#Sandbox
For playing around during development.

Todo:

- check that lookup table data is bound to the instance, not to the class

In [134]:
import glob
import os.path
import traceback
from parsimonious.exceptions import ParseError, IncompleteParseError
import pandas as pd

In [3]:
%pylab inline
import pysd
print pysd.__version__
print pysd.__file__

Populating the interactive namespace from numpy and matplotlib
0.3.2
pysd/__init__.pyc


In [10]:
test_dir = '../test-models/'
testfiles = glob.glob(test_dir+'*/*/*.mdl')

In [192]:
err_str = "\n"
threshold = 1

for modelfile in testfiles:
    directory = os.path.dirname(modelfile)
    try:
        model = pysd.read_vensim(modelfile)
        
        canon = pd.read_csv(directory+'/output.csv', index_col='Time')
        canon.columns = [pysd.builder.make_python_identifier(x) for x in canon.columns.values]
        
        output = model.run(return_columns=list(canon.columns.values), 
                           return_timestamps=canon.index.values)
        
        assert (canon-output).max().max() < 1
        
        print '.',
        
    except ParseError as e:
        print 'FP',
        
        err_str += '-'*60 + '\n'
        err_str += 'Parse Error in: %s \n'%modelfile
        err_str += 'At line: %i, column%i \n'%(e.line(), e.column())
        err_str += 'On rule: %s \n'%e.expr.name
        err_str += e.expr.__repr__() + '\n\n'
        err_str += e.text.splitlines()[e.line()-1] + '\n' #line numbers are 1 based, most likely
        err_str += '^'.rjust(e.column()) + '\n\n'
        
    except IOError as e:
        print 'E',
        
        err_str += '-'*60 + '\n'
        err_str += 'Failure loading canonical output for: %s \n'%modelfile 
        
    except AssertionError as e:
        print 'FC',
        
        err_str += '-'*60 + '\n'
        err_str += 'Model output does not match canon for: %s \n\n'%modelfile
        err_str += str((canon-output).max())
        err_str += '\n'

        

print err_str

. . E .Illegal input detected (internal error).
Run with full_output = 1 to get quantitative information.
 FC FP FP . FP FP . FC . FP .Illegal input detected (internal error).
Run with full_output = 1 to get quantitative information.
 FC FC FP . . . 
------------------------------------------------------------
Failure loading canonical output for: ../test-models/samples/Workforce/workforce.mdl 
------------------------------------------------------------
Model output does not match canon for: ../test-models/tests/builtin_max/builtin_max.mdl 

output   NaN
dtype: float64
------------------------------------------------------------
Parse Error in: ../test-models/tests/builtin_min/builtin_min.mdl 
At line: 3, column10 
On rule: Conditional 
<Sequence Conditional = "<=" / "<" / ">=" / ">" / "=" _ Term at 0x4399474000>

	min(Time, 5)
         ^

------------------------------------------------------------
Parse Error in: ../test-models/tests/delays/test_delays.mdl 
At line: 3, column22 
On 

In [187]:
(canon-output).

inflowa        0.000000e+00
stocka         3.552714e-15
test_arccos    5.307180e-06
test_arcsin    4.692820e-06
test_arctan    4.692820e-06
test_cos       4.966004e-07
test_sin       5.145580e-07
test_tan       4.586984e-05
dtype: float64

In [176]:
print model.components.state

{}


In [161]:
output.head()

inflowa  stocka  test_arccos  test_arcsin  test_arctan  test_cos  \
0.000        1 -10.000     2.566371     0.575222    -0.575222 -0.839072   
0.125        1  -9.875     2.691371     0.450222    -0.450222 -0.900351   
0.250        1  -9.750     2.816371     0.325222    -0.325222 -0.947580   
0.375        1  -9.625     2.941371     0.200222    -0.200222 -0.980022   
0.500        1  -9.500     3.066371     0.075222    -0.075222 -0.997172   

       test_sin  test_tan  
0.000  0.544021 -0.648361  
0.125  0.435165 -0.483329  
0.250  0.319519 -0.337195  
0.375  0.198887 -0.202941  
0.500  0.075151 -0.075364

In [150]:
canon = pd.read_csv(directory+'/output.csv')
list(canon.columns.values)

['Time',
 'Contact Infectivity',
 'Duration',
 'Infectious',
 'Recovered',
 'Recovering',
 'Succumbing',
 'Susceptible',
 'Total Population']

In [145]:
canon.columns.values

array(['time', 'contact_infectivity', 'duration', 'infectious',
       'recovered', 'recovering', 'saveper', 'succumbing', 'susceptible',
       'total_population'], dtype=object)

In [146]:
m = e.expr.members[-1]

In [133]:
m.members

[<Sequence Factor = ExpBase _ Multiplicative* at 0x4399571024>,
 <ZeroOrMore _ = spacechar* at 0x4401248128>,
 <ZeroOrMore Additive* at 0x4399573904>]

In [127]:
e.expr

<Sequence Conditional = "<=" / "<" / ">=" / ">" / "=" _ Term at 0x4399573344>

In [95]:
print e.text

{UTF-8}
output=
	min(Time, 5)
	~	
	~		|

********************************************************
	.Control
********************************************************~
		Simulation Control Parameters
	|

FINAL TIME  = 10
	~	Month
	~	The final time for the simulation.
	|

INITIAL TIME  = 0
	~	Month
	~	The initial time for the simulation.
	|

SAVEPER  = 
        TIME STEP 
	~	Month [0,?]
	~	The frequency with which output is stored.
	|

TIME STEP  = 1
	~	Month [0,?]
	~	The time step for the simulation.
	|

\\\---/// Sketch information - do not modify anything except names
V300  Do not put anything below this section - it will be ignored
*View 1
$192-192-192,0,Times New Roman|12||0-0-0|0-0-0|0-0-255|-1--1--1|-1--1--1|192,192,100,0
10,1,output,218,156,72,24,8,131,0,0,0,0,0,0
10,2,Time,218,216,53,19,8,2,0,3,-1,0,0,0,128-128-128,0-0-0,|0||128-128-128
1,3,2,1,0,0,0,0,0,64,0,-1--1--1,,1|(218,195)|
///---\\\
:L<%^E!@
9:Current
15:0,0,0,0,0,0
19:100,0
27:2,
34:0,
5:output
35:Date
36:YYYY-MM-DD
37

In [89]:
e.text.splitlines()

[u'{UTF-8}',
 u'Smooth3 output=',
 u'\tSMOOTH3(Input,Adjustment Time)',
 u'\t~\t',
 u'\t~\t\t|',
 u'',
 u'Smooth N output=',
 u'\tSMOOTH N(Input, Adjustment Time , 0 , 5 )',
 u'\t~\t',
 u'\t~\t\t|',
 u'',
 u'Smooth output=',
 u'\tSMOOTH(Input, Adjustment Time )',
 u'\t~\t',
 u'\t~\t\t|',
 u'',
 u'SmoothI output=',
 u'\tSMOOTHI(Input, Adjustment Time, 0 )',
 u'\t~\t',
 u'\t~\t\t|',
 u'',
 u'Adjustment Time=',
 u'\t2',
 u'\t~\t',
 u'\t~\t\t|',
 u'',
 u'Smooth3I output=',
 u'\tSMOOTH3I(Input, Adjustment Time, 0 )',
 u'\t~\t',
 u'\t~\t\t|',
 u'',
 u'Input=',
 u'\tSTEP(5 , 5)',
 u'\t~\t',
 u'\t~\t\t|',
 u'',
 u'********************************************************',
 u'\t.Control',
 u'********************************************************~',
 u'\t\tSimulation Control Parameters',
 u'\t|',
 u'',
 u'FINAL TIME  = 20',
 u'\t~\tMonth',
 u'\t~\tThe final time for the simulation.',
 u'\t|',
 u'',
 u'INITIAL TIME  = 0',
 u'\t~\tMonth',
 u'\t~\tThe initial time for the simulation.',
 u'\t|',
 

In [2]:
model = pysd.read_vensim('tests/vensim/Teacup.mdl')

#model = pysd.read_xmile('tests/xmile/teacup.xmile')

In [ ]:
model.run()

In [4]:
import pandas as pd

In [ ]:
pd.read_excel()

In [ ]:
model.set_components({'room_temperature':pd.Series(np.linspace(20,80,30)})

In [11]:
result = model.run()
result.head()

cuptemp
0.00  170.000000
0.25  165.755830
0.50  161.688256
0.75  157.789930
1.00  154.053813

In [12]:
result.plot()

## PySD Performance
Lets plot the history of our speedup efforts:

In [13]:
import pandas as pd

dictlist = []
with open('tests/speedtest_results.json') as infile:
    for line in infile:
        dictlist.append(eval(line))
    
speedtests = pd.DataFrame(dictlist)
speedtests['run teacup baseline'].plot(marker='o');